# Organizar de tal modo que no toque recalcular matrices!

In [ ]:
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import cv2
import cv2.cv as cv
import glob
import re
from matplotlib import pyplot as plt
import time as t
print "OpenCV version: {}".format(cv2.__version__)
# https://wiki.python.org/moin/PythonSpeed/PerformanceTips

In [ ]:
i = cv2.imread('stereo1/sample.png')
img_w = i.shape[1]
img_h = i.shape[0]
del i
RESOLUTION = (img_w, img_h)
grid_w = 7
grid_h = 7
GRID_SIZE = (grid_w, grid_h)
sq_size = 3 # cm
crit = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((grid_w*grid_h,3), np.float32)
objp[:,:2] = np.mgrid[0:grid_w,0:grid_h].T.reshape(-1,2) * sq_size

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12
fig_size[1] = 6
plt.rcParams["figure.figsize"] = fig_size

In [ ]:
imagesL = sorted(glob.glob('stereo1/img/L*.png'))
imagesR = sorted(glob.glob('stereo1/img/R*.png'))

objpoints = []
imgpoints1 = []
imgpoints2 = []

for fnameL, fnameR in zip(imagesL, imagesR):
    imgL = cv2.imread(fnameL)
    imgR = cv2.imread(fnameR)
    grayL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)
    retL, cornersL = cv2.findChessboardCorners(grayL, GRID_SIZE)
    retR, cornersR = cv2.findChessboardCorners(grayR, GRID_SIZE)
        
    if retL is True and retR is True:
        objpoints.append(objp)
        cv2.cornerSubPix(grayL, cornersL, (11,11), (-1,-1), crit)
        cv2.cornerSubPix(grayR, cornersR, (11,11), (-1,-1), crit)
        imgpoints1.append(cornersL)
        imgpoints2.append(cornersR)
        cv2.drawChessboardCorners(imgL, GRID_SIZE, cornersL, retL)
        cv2.drawChessboardCorners(imgR, GRID_SIZE, cornersR, retR)
    print retL, fnameL
    print retR, fnameR
    plt.subplot(1,2,1),plt.imshow(imgL)
    plt.subplot(1,2,2),plt.imshow(imgR)
    plt.show()
        
        
retL, mtxL, distL, rvecsL, tvecsL = cv2.calibrateCamera(objpoints, imgpoints1, RESOLUTION)
print mtxL
print distL

retR, mtxR, distR, rvecsR, tvecsR = cv2.calibrateCamera(objpoints, imgpoints2, RESOLUTION)
print mtxR
print distR

In [ ]:
imagesL = sorted(glob.glob('stereo1/scene/L*.png'))
imagesR = sorted(glob.glob('stereo1/scene/R*.png'))

ret, _, _, _, _, R, T, E, F  = cv2.stereoCalibrate(objpoints, imgpoints1, imgpoints2, RESOLUTION)
R1, R2, P1, P2, Q, _, _ = cv2.stereoRectify(mtxL, distL, mtxR, distR, RESOLUTION, R, T, flags=cv2.CALIB_ZERO_DISPARITY)

map1L, map2L = cv2.initUndistortRectifyMap(mtxL, distL, R1, mtxL, RESOLUTION, cv2.CV_16SC2)
map1R, map2R = cv2.initUndistortRectifyMap(mtxR, distR, R2, mtxR, RESOLUTION, cv2.CV_16SC2)

'''
for fnameL, fnameR in zip(imagesL, imagesR):
    nameL = re.findall(r'[\w\d\-]+\.\w+',fnameL)[0]
    nameR = re.findall(r'[\w\d\-]+\.\w+',fnameR)[0]
    imgL = cv2.imread(fnameL)
    imgR = cv2.imread(fnameR)    
    imgL = cv2.remap(imgL, map1L, map2L, cv2.INTER_CUBIC)
    imgR = cv2.remap(imgR, map1R, map2R, cv2.INTER_CUBIC)
    imgL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    imgR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)
    cv2.imwrite('stereo1/rect/'+nameL, imgL)
    cv2.imwrite('stereo1/rect/'+nameR, imgR)
    plt.subplot(1,2,1), plt.imshow(imgL, 'gray')
    plt.subplot(1,2,2), plt.imshow(imgR, 'gray')
    plt.show()
'''

In [ ]:
# np.savez('data/maps.npz', map1L, map2L, map1R, map2R)

In [ ]:
# import ipywidgets as widgets
from IPython.html import widgets
from IPython.display import display

# M = img_w / 8
M = 48
# P1 = 8*number_of_image_channels*SADWindowSize*SADWindowSize
# P2 = 32*number_of_image_channels*SADWindowSize*SADWindowSize

def stereoSGBM(mindisp=0, sadw=1, disp12=0, pfiltc=0, unqrat=5, spckw=0, spckr=1):
    imgL = cv2.imread('stereo1/scene/L0296.png')
    imgR = cv2.imread('stereo1/scene/R0296.png')
    imgL = cv2.remap(imgL, map1L, map2L, cv2.INTER_CUBIC)
    imgR = cv2.remap(imgR, map1R, map2R, cv2.INTER_CUBIC)
    numdisp = M - mindisp
    stereo = cv2.StereoSGBM(mindisp, M-mindisp, sadw, 8*sadw*sadw, 32*sadw*sadw, disp12, pfiltc, unqrat, spckw, spckr)
        
    grayL = cv2.cvtColor(imgL,cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(imgR,cv2.COLOR_BGR2GRAY)
    # grayL = cv2.Laplacian(grayL, cv2.CV_16S, ksize=1, scale=1, delta=0)
    # grayR = cv2.Laplacian(grayR, cv2.CV_16S, ksize=1, scale=1, delta=0)
    # grayL = cv2.convertScaleAbs(grayL, alpha=0.8, beta=100)
    # grayR = cv2.convertScaleAbs(grayR, alpha=0.8, beta=100)
    grayL = cv2.pyrDown(grayL)
    grayR = cv2.pyrDown(grayR)

    disparity = stereo.compute(grayL,grayR)
    plt.imshow(disparity)
    plt.show()
    plt.subplot(1,2,1),plt.imshow(grayL, 'gray')
    plt.subplot(1,2,2),plt.imshow(grayR, 'gray')
    # plt.subplot(2,2,3),plt.imshow(dstL, 'gray')
    # plt.subplot(2,2,4),plt.imshow(dstR, 'gray')
    # plt.show()
    
mindisp_slider = widgets.IntSlider(min=0, max=M-16, step=16, value=16, description='minDisparity')
sadw_slider = widgets.IntSlider(min=1, max=21, step=2, value=1, description='SADWindowSize')
disp12_slider = widgets.IntSlider(min=-1, max=5, step=1, value=1, description='disp12MaxDiff')
pfiltc_slider = widgets.IntSlider(min=0, max=63, step=1, value=54, description='preFilterCap')
unqrat_slider = widgets.IntSlider(min=5, max=15, step=1, value=145, description='uniquenessRatio')
spckw_slider = widgets.IntSlider(min=50, max=200, step=1, value=160, description='speckleWindowSize')
spckr_slider = widgets.IntSlider(min=1, max=2, step=1, value=2, description='speckleRange')

w = widgets.interactive(stereoSGBM, mindisp=mindisp_slider, sadw=sadw_slider, disp12=disp12_slider,
                        pfiltc=pfiltc_slider, unqrat=unqrat_slider, spckw=spckw_slider, spckr=spckr_slider)
display(w)

In [ ]:
nms = dir(cv2) # list of everything in the cv2 module
[m for m in nms if 'PROP' in m]

In [ ]:
cap = cv2.VideoCapture(0)
cap2 = cv2.VideoCapture(1)

# cap.set(cv.CV_CAP_PROP_FPS, 2)
# cap2.set(cv.CV_CAP_PROP_FPS, 2)
M = 96
m = 16
n = M - m

with np.load('data/maps.npz') as X:
    map1L, map2L, map1R, map2R = [X[i] for i in ('arr_0', 'arr_1', 'arr_2', 'arr_3')]

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    ret2, frame2 = cap2.read()

    imgL = cv2.remap(frame, map1L, map2L, cv2.INTER_LINEAR)
    imgR = cv2.remap(frame2, map1R, map2R, cv2.INTER_LINEAR)


    stereo = cv2.StereoSGBM(m, n, 1, 8, 32, 3, 41, 14, 200, 1)

    disparity = stereo.compute(imgL,imgR)
    plt.imshow(disparity)
    plt.show()

In [ ]:
help(cv2.adaptiveThreshold)